In [1]:
import requests
import pandas as pd
import json
import spacy
import numpy as np
from collections import deque

In [121]:
api_base = "https://api.semanticscholar.org/v1/paper/"
arxiv_id = "arXiv:1703.06870"
res = requests.get(api_base + arxiv_id) # be able to do this by doi as well, whatever.
base_data = json.loads(res.text)

In [122]:
list(base_data.keys())

['abstract',
 'arxivId',
 'authors',
 'citationVelocity',
 'citations',
 'corpusId',
 'doi',
 'fieldsOfStudy',
 'influentialCitationCount',
 'is_open_access',
 'is_publisher_licensed',
 'paperId',
 'references',
 'title',
 'topics',
 'url',
 'venue',
 'year']

In [124]:
count = 0
for each in base_data['citations']:
    if each['isInfluential'] == True:
        count +=1;


In [126]:
count = 0
for each in base_data['references']:
    if each['isInfluential'] == True:
        count +=1;


In [183]:
base_data['paperId']

'ea99a5535388196d0d44be5b4d7dd02029a43bb2'

In [182]:
count

6

In [10]:
np.zeros(3)

array([0., 0., 0.])

In [137]:
a = [('2',1),('1',2)]

In [138]:
a.sort(key=lambda tup: tup[1], reverse=True)


In [139]:
for each in range (3):
    a.append(('',0))

In [140]:
print(len(a))

5


In [152]:
a = [(1,2,3)]

In [153]:
b, c, d = a[0]

2

In [141]:
a

[('1', 2), ('2', 1), ('', 0), ('', 0), ('', 0)]

In [144]:
a[:breadth]

[('1', 2), ('2', 1), ('', 0)]

In [56]:
a = deque(a)

In [65]:
b = a.popleft()
print(a)
print(b)

deque([])
('', 0)


In [70]:
q

deque([])

In [69]:
q = deque()

In [192]:
# influential references
# find number of citations
# get reference data from top 3 citations
# add to layer
#if link is highly influential, get it
#if infcitcount within top n citations add link to tuple ,
global q, breadth

refdict = {"arxivId":[],"authors":[],"citationVelocity":[],"num_citations":[],
           "corpusId":[],"doi":[],"fieldsOfStudy":[], "influentialCitationCount":[],
           "is_open_access":[], "is_publisher_licensed":[],"paperId":[],"num_references":[],
           "title":[],"topics":[],"url":[],"venue":[],"year":[], "parent": []}
    

def add_record(res, parent):
    refdict['arxivId'].append(res['arxivId'])
    refdict['authors'].append(res['authors'])
    refdict['citationVelocity'].append(res['citationVelocity'])
    refdict['num_citations'].append(len(res['citations']))
    refdict['corpusId'].append(res['corpusId'])
    refdict['doi'].append(res['doi'])
    refdict['fieldsOfStudy'].append(res['fieldsOfStudy'])
    refdict['influentialCitationCount'].append(res['influentialCitationCount'])
    refdict['is_open_access'].append(res['is_open_access'])
    refdict['paperId'].append(res['paperId'])
    refdict['is_publisher_licensed'].append(res['is_publisher_licensed'])
    refdict['num_references'].append(len(res['references']))
    refdict['title'].append(res['title'])
    refdict['topics'].append(res['topics'])
    refdict['url'].append(res['url'])
    refdict['venue'].append(res['venue'])
    refdict['year'].append(res['year'])
    refdict['parent'].append(parent)
    return len(refdict['parent'])-1

def resolve_accessor(res):
    if res['arxivId'] == None:
        if res['paperId'] == None:
            if res['doi'] == None:
                return(None)
            else:
                return(res['doi'])
        else:
            return(res['paperId'])
    else:
        return('arxiv:' + res['arxivId'])

q = deque()
breadth = 3
max_depth = 2
doi = "arXiv:1703.06870"
def find_qualified_children(access, depth=0, parent =-1):
    print('depth =', depth)
    res = json.loads(requests.get(api_base+access).text)
    parent = add_record(res, parent)
    if depth == max_depth:
        try:
            nextaccess, _, depth, parent = q.popleft()
            find_qualified_children(nextaccess, depth, parent)
        except:
            return
        return
    topnchildren = [('',-1)]*breadth
    for ref in res['references']:
        if ref['isInfluential'] == True:
            child_accessor = resolve_accessor(ref)
            if child_accessor == None:
                print("skipped")
                continue
            child = json.loads(requests.get(api_base+child_accessor).text)
            try:
                icc = child['influentialCitationCount']
            except KeyError:
                print("not in s2 database")
            topnchildren.append((child_accessor, icc, depth+1, parent))
            topnchildren.sort(key=lambda tup: tup[1], reverse=True)
            topnchildren = topnchildren[:breadth]
    print('topn =', topnchildren, 'q length = ', len(q))
    for each in topnchildren:
        if each[0] != '':
            print(f"appended {each[0]}")
            q.append(each)
    nextaccess, _, depth, parent = q.popleft() 
    find_qualified_children(nextaccess, depth, parent)
    if len(q) == 0:
        return
    


In [193]:
find_qualified_children(doi)

depth = 0
topn = [('10d85561e4aafc516d10064f30dff05b41f70afe', 2743, 1, 0), ('arxiv:1604.01685', 685, 1, 0), ('arxiv:1812.08008', 396, 1, 0)] q length =  0
appended 10d85561e4aafc516d10064f30dff05b41f70afe
appended arxiv:1604.01685
appended arxiv:1812.08008
depth = 1
topn = [('', -1), ('', -1), ('', -1)] q length =  2
depth = 1
topn = [('arxiv:1506.01497', 3365, 2, 2), ('9201bf6f8222c2335913002e13fbac640fc0f4ec', 1924, 2, 2), ('arxiv:1405.0312', 1584, 2, 2)] q length =  1
appended arxiv:1506.01497
appended 9201bf6f8222c2335913002e13fbac640fc0f4ec
appended arxiv:1405.0312
depth = 1
topn = [('arxiv:1512.03385', 8538, 2, 3), ('arxiv:1605.03170', 46, 2, 3), ('arxiv:1511.06645', 35, 2, 3)] q length =  3
appended arxiv:1512.03385
appended arxiv:1605.03170
appended arxiv:1511.06645
depth = 2
depth = 2
depth = 2
depth = 2
depth = 2
depth = 2


In [194]:
refs = pd.DataFrame.from_dict(refdict)

In [201]:
refs.to_csv("graph.csv", index = False)

In [147]:
def recurse (level = 0):
    print(level)
    if level == 3:
        return
    for each in range (3):
        recurse(level+1)
        

In [148]:
recurse()

0
1
2
3
3
3
2
3
3
3
2
3
3
3
1
2
3
3
3
2
3
3
3
2
3
3
3
1
2
3
3
3
2
3
3
3
2
3
3
3
